# Pré-processamento e separação

Esse notebook irá pré-processar e separar os dados.

Entrada: banco de dados "breath.db"
Saída: arquivos com os dados processados e separados, em DataFrames serializados e arquivos CSV

Antes de começar, verifique se você possui todas as bibliotecas necessárias no segundo conjunto da próxima célula.

Caso não possua, instale utilizando `python -m pip install <nome_da_biblioteca>` (Windows) ou `pip3 install <nome_da_biblioteca>` (Linux).

In [1]:
import sqlite3
import json
import pickle

import pandas as pd
import numpy as np
from geopy.distance import distance as geodist

## Leitura do banco de dados

O banco de dados será lido, e será extraído metadados (nomes das tabelas, esquemas das tabelas).

In [2]:
conn = sqlite3.connect('breath.db')
cursor = conn.cursor()

In [3]:
cursor.execute("CREATE TABLE IF NOT EXISTS Workflow( Nome TEXT PRIMARY KEY, Executado BOOL NOT NULL DEFAULT '0')")

In [4]:
data = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

table_names = []

for i in data.fetchall():
    if i[0] != "sqlite_sequence":
        table_names.append(i[0])

table_names

['Sintomas',
 'Cidades',
 'Estacoes',
 'Clima',
 'SRAG',
 'Estacao_Cidade',
 'Workflow']

In [5]:
table_columns = {}

for name in table_names:
    data = cursor.execute("SELECT * FROM {0}".format(name))
    table_columns[name] = []

    for column in data.description:
        table_columns[name].append(column[0])

In [6]:
table_schemas = {}

for name in table_columns:
    schema = name+"("

    for column in table_columns[name]:
        schema += column + ", "
    schema += ")"

    table_schemas[name] = schema

In [7]:
for name in table_schemas:
    print(table_schemas[name])
    print()

Sintomas(Id, Tipo, Ano, Mês, Dia, Cidade, )

Cidades(Id, UF, Nome_UF, Mesorregiao_geografica, Nome_mesorregiao, Microrregiao_geografica, Nome_microrregiao, Municipio, Cod_municipio, Nome_municipio, Pop_estimada, lat, lon, )

Estacoes(Id, Estacao, Regiao, UF, Codigo, Prim_data, alt, lon, lat, )

Clima(id, date, station, precipitacao, pressao_at_max, pressao_at_min, radiacao, temp_max, temp_min, umidade, max_vent, velocidade_vent, region, state, lat, lon, elvt, )

SRAG(id, DT_NOTIFIC, ID_MUNICIP, SEM_NOT, SG_UF_NOT, DT_SIN_PRI, DT_NASC, NU_IDADE_N, CS_SEXO, CS_GESTANT, CS_RACA, CS_ESCOL_N, SG_UF, ID_MN_RESI, ID_OCUPA_N, VACINA, FEBRE, TOSSE, CALAFRIO, DISPNEIA, GARGANTA, ARTRALGIA, MIALGIA, CONJUNTIV, CORIZA, DIARREIA, OUTRO_SIN, OUTRO_DES, CARDIOPATI, PNEUMOPATI, RENAL, HEMOGLOBI, IMUNODEPRE, TABAGISMO, METABOLICA, OUT_MORBI, MORB_DESC, HOSPITAL, DT_INTERNA, CO_UF_INTE, CO_MU_INTE, DT_PCR, PCR_AMOSTR, PCR_OUT, PCR_RES, PCR_ETIOL, PCR_TIPO_H, PCR_TIPO_N, DT_CULTURA, CULT_AMOST, CULT_OUT,

## Processamento de dados

Aqui, os dados do banco serão transformados e aglomerados em visões até obtemos a tabela final desejada para o dataset

### Processamento de dados climáticos

Nessa seção, os dados climáticos serão processados, realizando as seguintes operações:

- Será criado uma tabela relacionando as estações climáticas com as cidades. Uma estação é associada a uma cidade se ela está até 25 km da posição geográfica registrada para a cidade
- Será criado uma visão com os dados climáticos associados com o dia desde a epoch
- A tabela de dados climáticos é associada com as cidades, utilizando a tabela que relaciona as estações com as cidades criada anteriormente. O dado climático de uma cidade é definido como a média dos dados das estações associadas.

In [8]:
# Carrega e salva posições e nomes das estações climáticas e cidades

data = cursor.execute("SELECT lat, lon FROM Estacoes")
data = data.fetchall()
pos_estacoes = data 

data = cursor.execute("SELECT Estacao FROM Estacoes")
data = data.fetchall()
nome_estacoes = np.asarray(data).flatten()

data = cursor.execute("SELECT Id FROM Estacoes")
data = data.fetchall()
id_estacoes = np.asarray(data).flatten()

data = cursor.execute("SELECT lat, lon FROM Cidades")
data = data.fetchall()
pos_cidades = data

data = cursor.execute("SELECT Nome_municipio FROM Cidades")
data = data.fetchall()
nome_cidades = np.asarray(data).flatten()

data = cursor.execute("SELECT Id FROM Cidades")
data = data.fetchall()
id_cidades = np.asarray(data).flatten()

In [9]:
# Cria tabela que relaciona estações climáticas com cidades

query = "CREATE TABLE IF NOT EXISTS Estacao_Cidade(Id_Cidade INTEGER, Id_Estacao INTEGER, "
query += "FOREIGN KEY (Id_Cidade) REFERENCES Cidades(Id), FOREIGN KEY (Id_Estacao) REFERENCES Estacoes(Id)"
query += ")"
cursor.execute(query)

In [10]:
# Procura as estações climáticas mais próximas (distancia < 25 km)

for i in range(len(pos_cidades)):

    distances = np.ndarray(len(pos_estacoes), np.float32)

    for j in range(len(pos_estacoes)):
        distances[j] = geodist(pos_cidades[i], pos_estacoes[j]).km

    indexes = np.argwhere(distances < 25.0).flatten()

    id_cidade_to_insert = id_cidades[i]
    id_estacoes_to_insert = id_estacoes[indexes]

    for id_estacao in id_estacoes_to_insert:
        query = "INSERT INTO Estacao_Cidade(Id_Cidade, Id_Estacao) VALUES('{0}', '{1}')".format(id_cidade_to_insert, id_estacao)
        cursor.execute(query)
    
conn.commit()

In [22]:
# Cria view com dia desde a epoch para a tabela de dados climáticos

query = "CREATE VIEW IF NOT EXISTS Clima_Data  AS SELECT *, "
query += "(strftime('%s', substr(date, 7, 8) ||'-'|| substr(substr(date, 4, 6),1,2) ||'-'|| substr(date, 1, 2) )/86400) AS DIA "
query += "FROM Clima"

cursor.execute(query)

In [55]:
# View com o clima e as cidades

query = "CREATE VIEW IF NOT EXISTS Clima_Cidade AS " 
query += "SELECT Nome_municipio, Cidades.Pop_estimada, Clima_Data.DIA, "
query += "Avg(precipitacao) AS Precipitacao, Avg(pressao_at_max) AS Pressao_at_max, Avg(pressao_at_min) AS Pressao_at_min, Avg(radiacao) AS Radiacao, Avg(temp_max) AS Temp_max, Avg(temp_min) AS Temp_min, Avg(umidade) AS Umidade, Avg(max_vent) AS Max_vent, Avg(velocidade_vent) AS Velocidade_vent "
query += "FROM Cidades, Estacao_Cidade, Estacoes, Clima_Data " 
query += "WHERE Estacao_Cidade.Id_Cidade=Cidades.Id AND Estacoes.Id = Estacao_Cidade.Id_Estacao AND Estacoes.Estacao = Clima_Data.station "
query += "GROUP BY Nome_municipio, DIA"

cursor.execute(query)
conn.commit()

### Processamento de dados médicos

Aqui, os dados do SRAG serão processados, realizando as seguintes operações:

- As datas dos dados serão transformadas para o formato de dia desde a epoch
- Os casos por cidade por dia serão aglomerados, considerando um caso todas as entradas da tabela que possuem febre, tosse ou dor de garganta. A quantidade de casos pela população da cidade também é calculada.

In [12]:
# Cria view com dia desde epoch para a tabela de dados de sintoma

query = "CREATE VIEW IF NOT EXISTS SRAG_Data AS SELECT *, "
query += "(strftime('%s', substr(DT_NOTIFIC, 7, 8) ||'-'|| substr(substr(DT_NOTIFIC, 4, 6),1,2) ||'-'|| substr(DT_NOTIFIC, 1, 2) )/86400) AS DIA "
query += "FROM SRAG"


cursor.execute(query)

In [15]:
# Cria view de casos por cidade e dia

query = "CREATE VIEW IF NOT EXISTS Casos_Dia AS "
query += "SELECT ID_MUNICIP, DIA, Count(*) AS Casos, Count(*)/Pop_estimada as Casos_pela_pop, Pop_estimada "
query += "FROM SRAG_Data, Cidades "
query += "WHERE SRAG_Data.ID_MUNICIP = Cidades.Nome_municipio AND(FEBRE='1.0' OR TOSSE='1.0' OR GARGANTA='1.0') "
query += "GROUP BY ID_MUNICIP, DIA"
cursor.execute(query)

### Geração da tabela final

A tabela final do dataset relacionando o clima aos casos é criada e lida do banco de dados.

In [62]:
# Cria tabela final que relaciona o clima com os casos

query = "CREATE VIEW IF NOT EXISTS Clima_Casos AS "
query += "SELECT Clima_Cidade.Nome_municipio, Clima_Cidade.DIA as Dia, Precipitacao,  "
query += "Pressao_at_max, Pressao_at_min, Radiacao, Temp_max, Temp_min, Umidade, Max_vent, Velocidade_vent, "
query += "Clima_Cidade.Pop_estimada, IFNULL(Casos, 0) as Casos, IFNULL(Casos_pela_pop, 0) AS Casos_pela_pop "
query += "FROM Clima_Cidade LEFT JOIN Casos_Dia "
query += "ON Clima_Cidade.Nome_municipio = Casos_Dia.ID_MUNICIP AND Clima_Cidade.DIA = Casos_Dia.DIA "
query += "ORDER BY Nome_municipio, Clima_Cidade.DIA "

data = cursor.execute(query)
data = data.fetchall()

conn.commit()

In [65]:
# Solicita todos os dados

data_execution = cursor.execute("SELECT * FROM Clima_Casos")
data = data_execution.fetchall()

final_data = data

In [68]:
# Pega a sequência das colunas

final_column_names = []

for column in data_execution.description:
    final_column_names.append(column[0])

final_column_names

['Nome_municipio',
 'Dia',
 'Precipitacao',
 'Pressao_at_max',
 'Pressao_at_min',
 'Radiacao',
 'Temp_max',
 'Temp_min',
 'Umidade',
 'Max_vent',
 'Velocidade_vent',
 'Pop_estimada',
 'Casos',
 'Casos_pela_pop']

## Separação e exportação dos dados

Aqui os dados são separados nos conjuntos de treino, validação e teste, para treino dos modelos e do ensemble.

### Estratégia de separação

Seguimos a seguinte proporção de divisão:

- 10% Teste
- 70% Para os modelos
- 20% Para o ensemble
  
Os dados para os modelos e para o ensemble são divididos em:

- 80% Treino
- 20% Validação

Portanto, a divisão final é:

- 10% Teste
- 56% Treino dos modelos
- 14% Validação dos modelos
- 16% Treino do ensemble
-  4% Validação do ensemble

In [70]:
# Cria um DataFrame com os dados

final_dataframe = pd.DataFrame(final_data, columns = final_column_names)

final_dataframe.describe()

,Dia,Precipitacao,Pressao_at_max,Pressao_at_min,Radiacao,Temp_max,Temp_min,Umidade,Max_vent,Velocidade_vent,Casos,Casos_pela_pop
count,6.597097e+06,6.597097e+06,6.597097e+06,6.597097e+06,6.597097e+06,6.597097e+06,6.597097e+06,6.597097e+06,6.597097e+06,6.597097e+06,6.597097e+06,6.597097e+06
mean,1.615627e+04,2.295044e+00,9.631145e+02,9.625248e+02,1.483463e+03,2.859422e+01,1.845028e+01,6.579338e+01,8.806669e+00,2.432381e+00,1.987723e-02,4.874491e-03
std,1.570813e+03,7.498920e+00,3.760641e+01,3.758300e+01,1.612715e+03,5.032088e+00,4.562282e+00,1.535542e+01,2.985992e+00,1.308411e+00,4.686740e-01,1.577920e-01
min,1.108400e+04,0.000000e+00,7.498000e+02,7.487000e+02,0.000000e+00,-4.700000e+00,-9.000000e+00,7.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.491600e+04,0.000000e+00,9.338786e+02,9.333154e+02,1.025692e+03,2.570000e+01,1.590000e+01,5.575000e+01,6.900000e+00,1.538462e+00,0.000000e+00,0.000000e+00
50%,1.623800e+04,0.000000e+00,9.641333e+02,9.635364e+02,1.416083e+03,2.930000e+01,1.920000e+01,6.692308e+01,8.600000e+00,2.242857e+00,0.000000e+00,0.000000e+00
75%,1.750300e+04,4.000000e-01,9.969308e+02,9.962818e+02,1.730667e+03,3.210000e+01,2.170000e+01,7.700000e+01,1.050000e+01,3.150000e+00,0.000000e+00,0.000000e+00
max,1.874700e+04,2.990000e+02,1.050000e+03,1.049600e+03,4.426500e+04,4.500000e+01,4.480000e+01,1.000000e+02,8.840000e+01,2.379231e+01,1.910000e+02,3.217158e+01


In [90]:
# Separa os dados

n = len(final_dataframe)

test_df = final_dataframe[0:int(0.1*n)]
train_df = final_dataframe[int(0.1*n):int(0.66*n)]
val_df = final_dataframe[int(0.66*n):int(0.8*n)]
ens_train_df = final_dataframe[int(0.8*n):int(0.96*n)]
ens_val_df = final_dataframe[int(0.96*n):]

In [117]:
# Cria um dicionário com os dados para facilitar a exportação

df_dict = {}
df_dict["test_df"] = test_df
df_dict["train_df"] = train_df
df_dict["val_df"] = val_df
df_dict["ens_train_df"] = ens_train_df
df_dict["ens_val_df"] = ens_val_df

In [119]:
# Exporta os dados usando o pickle

for df_name in df_dict:
    file = open(df_name, 'wb')
    pickle.dump(df_dict[df_name], file)
    file.close()

file = open("dataset", 'wb')
pickle.dump(df_dict, file)
file.close()

In [131]:
# Exporta os dados para CSV

for df_name in df_dict:
    df_dict[df_name].to_csv(df_name+".csv")

In [129]:
# Exporta um arquivo descrevendo o dataset

info_dict = {}
info_dict["column_names"] = final_column_names
info_dict["feature_names"] = final_column_names[:-2]
info_dict["target_names"] = final_column_names[-2:]
info_dict["data_count"] = n
info_dict["train_count"] = len(train_df)
info_dict["test_count"] = len(test_df)
info_dict["val_count"] = len(val_df)
info_dict["ensemble_train_count"] = len(ens_train_df)
info_dict["ensemble_val_count"] = len(ens_val_df)

file = open("info.json", "w")
json.dump(info_dict, file)
file.close()

In [120]:
# Exemplo de importação

file = open("test_df", 'rb')
pickle.load(file)
file.close()

,Nome_municipio,Dia,Precipitacao,Pressao_at_max,Pressao_at_min,Radiacao,Temp_max,Temp_min,Umidade,Max_vent,Velocidade_vent,Pop_estimada,Casos,Casos_pela_pop
0,abaira,14004,0.0,874.530000,874.060000,2098.900000,27.0,19.3,47.100000,6.9,1.900000,8.681,0,0.0
1,abaira,14005,0.0,873.525000,873.066667,1822.916667,28.1,15.8,50.250000,8.0,2.383333,8.681,0,0.0
2,abaira,14006,0.0,873.841667,873.375000,1385.833333,26.2,18.7,60.250000,7.1,2.600000,8.681,0,0.0
3,abaira,14007,0.2,874.016667,873.516667,1137.250000,23.3,17.6,76.083333,7.6,2.275000,8.681,0,0.0
4,abaira,14008,0.0,873.816667,873.400000,926.666667,21.9,16.9,77.750000,6.0,1.625000,8.681,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659704,bela cruz,18184,0.0,1005.030769,1004.453846,1734.307692,33.3,23.4,63.307692,11.8,5.084615,32.851,0,0.0
659705,bela cruz,18185,0.0,1005.161538,1004.476923,1561.923077,32.6,24.3,64.846154,13.1,5.315385,32.851,0,0.0
659706,bela cruz,18186,0.0,1005.323077,1004.638462,1783.923077,34.8,23.0,54.846154,13.1,5.576923,32.851,0,0.0
659707,bela cruz,18187,0.0,1005.307692,1004.661538,1617.076923,33.7,24.0,61.076923,12.7,5.530769,32.851,0,0.0
